In [2]:
# Import statements
import string
import numpy as np
import pandas as pd
import scipy
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [24]:

df = pd.read_csv('pos-eng-5000.data.csv')
v = LabelEncoder()
everything = df[['a1', 'a2', 'a3', 'a4', 'a5', 'a6']].to_numpy()
word_list = []
for row in everything:
    for word in row:
        if word not in word_list:
            word_list.append(word)
            
class_code = v.fit_transform(df[['class']])
class_inverted = v.inverse_transform((class_code))

word_code = v.fit_transform(word_list)
word_inverted = v.inverse_transform(word_code)

word_code_dict = {}
for i in range(len(word_inverted)):
    word_code_dict[word_inverted[i]] = word_code[i]

grand_list = []
for row in everything:
    local_list = []
    for i in row:
        local_list.append(word_code_dict.get(i))
    grand_list.append(local_list)

X = scipy.sparse.csr_matrix(grand_list)
Y = scipy.sparse.csr_matrix(class_code)

/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
# Define a function to take as input training and testing vectors and labels
# Allow this to be extensible to let multiple classifiers be used here
def buildClassifiers(clf, X_train, X_test, y_train, y_test):

    clf.fit(X_train.todense(), y_train)
    y_pred = clf.predict(X_test.todense())

    f1 = f1_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    accuracy = accuracy_score(y_test, y_pred)
    return f1, precision, recall, accuracy

In [26]:
# Construct the classifiers at hand prior to folding the data through them
names = ['Naive_Bayes', 'Decision_Tree']
classifiers =   [GaussianNB(), 
                DecisionTreeClassifier(random_state=0)]
for name, clf in zip(names, classifiers):

    print('Now classifying', name)

    # Fold the data 5 times
    kf = KFold(n_splits = 5)
    foldCounter = 0
    aList, bList, cList, dList = list(), list(), list(), list()
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train = []
        y_test = []
        for i in train_index:
            y_train.append(Y[0, i])
        for i in test_index:
            y_test.append(Y[0, i])
        f1, precision, recall, accuracy = buildClassifiers(clf, X_train, X_test, y_train, y_test)
        aList.append(f1)
        bList.append(precision)
        cList.append(recall)
        dList.append(accuracy)

    print("\tAverage Macro F1 for {}:\t\t".format(name), np.mean(aList))
    print("\tAverage Macro Precision for {}:\t".format(name), np.mean(bList))
    print("\tAverage Macro Recall for {}:\t\t".format(name), np.mean(cList))
    print("\tAverage Macro Recall for {}:\t\t".format(name), np.mean(dList))



Now classifying Naive_Bayes


/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-pack

	Average Macro F1 for Naive_Bayes:		 0.19668173921969434
	Average Macro Precision for Naive_Bayes:	 0.21158628254506473
	Average Macro Recall for Naive_Bayes:		 0.22218919116342276
	Average Macro Recall for Naive_Bayes:		 0.3284
Now classifying Decision_Tree


/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-pack

	Average Macro F1 for Decision_Tree:		 0.5482373577735495
	Average Macro Precision for Decision_Tree:	 0.5648926214768967
	Average Macro Recall for Decision_Tree:		 0.5579069769121479
	Average Macro Recall for Decision_Tree:		 0.612


/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongshuo/anaconda3/lib/python3.6/site-pack